# Install spaCy

In [1]:
!pip install spacy
!python -m spacy download nl_core_news_sm
%matplotlib inline
!pip install pyLDAvis

     |████████████████████████████████| 10.6MB 4.1MB/s eta 0:00:01
     |████████████████████████████████| 194kB 17.2MB/s eta 0:00:01
     |████████████████████████████████| 122kB 28.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/4a/1c/6359be64e8301b84160f6f6f7936bbfaaa5e9a4eab6cbc681db07600b949/tqdm-4.45.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6c/f9/9a5658e2f56932e41eb264941f9a2cb7f3ce41a80cb36b2af6ab78e2f8af/catalogue-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/21/e1/e4e7b754e6be3a79c400eb766fb34924a6d278c43bb828f94233e0124a21/wasabi-0.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/86/85/40b8f66c2dd8f4fd9f09d59b22720cffecf1331e788b8a0cab5bafb353d1/plac-1.1.3-py2.py3-none-any.whl
     |████████████████████████████████| 3.7MB 18.2MB/s eta 0:00:01     |█████████▊                      | 1.1MB 18.2MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 

  Created wheel for nl-core-news-sm: filename=nl_core_news_sm-2.2.5-cp36-none-any.whl size=15403032 sha256=f1edfb556e623043acc7c80f27a9b67acbe2666a6e7a547dd9f73cf87f807fee
  Stored in directory: /tmp/pip-ephem-wheel-cache-4bwwgqjy/wheels/2b/1c/b7/262800297e4c1d763b1e087f579748288f84b76e9d43c6b11d
Successfully built nl-core-news-sm
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('nl_core_news_sm')
     |████████████████████████████████| 1.6MB 4.0MB/s eta 0:00:01
     |████████████████████████████████| 552kB 30.2MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=100203 sha256=65c71655d3717f0a0e1704a7523163f4e5cf231a4c42b8060fe57ba98bb2080a
  Stored in directory: /home/nbuser/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=35130 sha256=

# Import packages

In [2]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.nl.stop_words import STOP_WORDS
import nl_core_news_sm

from tqdm.notebook import tqdm

import pyLDAvis
import pyLDAvis.gensim

# Read data

In [3]:
#data = pd.read_csv('emerce_articles_small.csv')
data = pd.read_csv('emerce_articles.csv')
data.articles.count()

10490

In [4]:
data = data.dropna().reset_index(drop=True)
data.articles = [w.replace('[[\'', '').replace('[\'', '').replace('\'],', '').replace('\']]','') for w in data.articles]
data.articles = data.articles.replace('', np.nan)
data = data.dropna().reset_index(drop=True)
data.articles.head()

0    De verwerker van online en mobiele betalingen ...
1    De volledige Travix-organisatie (Vliegwinkel.n...
2    Software AG voegt voorspellende Big Data analy...
3    De Nederlandse broedmachine voor startende bed...
4    De juristen van Google krijgen een paar weken ...
Name: articles, dtype: object

# Cleaning

In [5]:
# Load dutch Language
nlp = spacy.load('nl_core_news_sm')

In [61]:
# My list of stop words.(should be reconsidered for each domain)
stop_list = ['gaan', 'komen', 'bedrijf', 'procent', 'jaar', 'willen', 'zeggen', 'krijgen', 'maken', 'zien', 'klant', 'toekomst', 'marketing', 'consument', 'zitten', 'euro',
            'beginnen', 'website', 'emerce', 'naam', 'gebruiken', 'lijken', 'aantal', 'v', 'stellen', 'vinden', 'laten', 'blijken', 'staan', '', 'bieden', 'aantal', 'geval',
            'volgen', 'bieden', 'kijken', 'hoeven', 'dag', 'week', 'liggen', 'aantal', 'staan', 'nemen', 'laten', 'blijken', 'deel', 'blijven', 'volgen', 'stellen', 'draaien', 'onderdeel', 'maand']

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [62]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # And use PoS tagging to keep only NOUN and VERB
    doc = [token.lemma_ for token in doc if token.pos_ == 'NOUN' or token.pos_ == 'VERB']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

ValueError: [E007] 'lemmatizer' already exists in pipeline. Existing names: ['tagger', 'parser', 'ner', 'lemmatizer', 'stopwords']

In [ ]:
tokens = []
# Iterates through each article in the corpus.
for doc in tqdm(data.articles):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    tokens.append(pr)

# LDA

## Data preperation

### Prepare bi-grams and tri-grams

In [63]:
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])

In [64]:
tokens[0]

['verwerker_betaling',
 'multisafepay',
 'tekenen_contract',
 'partner',
 'paysquare',
 'samenwerking_partij',
 'opkomst_kanaal',
 'noodzaak',
 'werkrelatie',
 'passen',
 'tijd',
 'leggen',
 'equen',
 'zorgen',
 'betaling_ontvangen',
 'merk',
 'betaalkaart',
 'europa',
 'werken',
 'contract',
 'richten',
 'winkel',
 'onderneming',
 'zaak',
 'accepteren',
 'betaalkaart',
 'multisafepay',
 'paysquare',
 'servicemodel',
 'afspreken',
 'betaling']

### Prepare objects for LDA gensim implementation

In [65]:
# Creates, which is a mapping of word IDs to words.
dictionary = corpora.Dictionary(tokens)
dictionary.filter_extremes(no_below=3)

# Turns each document into a bag of words.
corpus = [dictionary.doc2bow(tok) for tok in tokens]

### Running LDA

In [66]:
num_topics = 20
#lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                           id2word=dictionary,
#                                           num_topics=num_topics, 
#                                           random_state=2,
#                                           update_every=1,
#                                           passes=10,
#                                           alpha='auto')
#np.random.seed(123456)
lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary, \
                                  passes=10, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary.keys()))

## LDA Results

### Topics

In [67]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20):
    print(str(i)+": "+ topic)
    print()

0: 0.047*"apple" + 0.046*"app" + 0.023*"gebruiker" + 0.017*"apps" + 0.010*"lanceren" + 0.010*"werken" + 0.009*"mogelijkheid" + 0.008*"betalen" + 0.008*"io" + 0.007*"ontwikkelen" + 0.005*"applicatie" + 0.005*"brengen" + 0.005*"dienst" + 0.005*"gebruik" + 0.005*"versie" + 0.004*"informatie" + 0.004*"toevoegen" + 0.004*"ipad" + 0.004*"product" + 0.004*"introduceren"

1: 0.075*"winkel" + 0.040*"webwinkel" + 0.029*"product" + 0.025*"retailer" + 0.019*"webshop" + 0.015*"bol.com" + 0.014*"artikel" + 0.014*"nederland" + 0.012*"bestelling" + 0.012*"online" + 0.012*"verkopen" + 0.011*"openen" + 0.009*"assortiment" + 0.009*"kopen" + 0.009*"e-commerce" + 0.009*"leveren" + 0.008*"site" + 0.008*"aanbod" + 0.008*"verkoop" + 0.007*"kleding"

2: 0.032*"apple" + 0.024*"samsung" + 0.022*"telefoon" + 0.020*"smartphones" + 0.020*"tablet" + 0.017*"toestel" + 0.014*"apparaat" + 0.014*"verkopen" + 0.012*"smartphone" + 0.012*"fabrikant" + 0.011*"scherm" + 0.011*"iphone" + 0.011*"model" + 0.011*"markt" + 0.011*

## Allocating topics to documents

In [68]:
print(data.articles.loc[0])

De verwerker van online en mobiele betalingen Multisafepay tekent een nieuw contract met zijn partner Paysquare.  Na tien jaar samenwerking zien de partijen door de opkomst van meer en nieuwe kanalen de noodzaak om de werkrelatie aan te passen aan deze tijd en dat vast te leggen. Paysquare, onderdeel van Equens, zorgt ervoor dat bedrijven betalingen kunnen ontvangen met vrijwel alle merken betaalkaarten in Europa. Het bedrijf werkt voor 120.000 klanten.  Het nieuwe contract richt zich op hybride winkels, ondernemingen die zowel een fysieke als ook een online zaak hebben. “Naast het veilig accepteren van internationale betaalkaarten via MultiSafepay en PaySquare,is er nu ook een servicemodel afgesproken voor betalingen buiten webwinkels.”


In [69]:
lda_model[corpus[0]]

[(1, 0.09587572), (16, 0.77010036), (19, 0.12776619)]

#### Predicting topics on unseen documents

In [70]:
new_doc = '''De content clean-up: zeg meer met minder
Je herkent het vast wel. In de loop der jaren is je website steeds groter geworden. Je redactie heeft meer en meer pagina’s te beheren, en er komt dagelijks content bij. Hierdoor wordt het contentbeheer met de dag arbeidsintensiever. Redacteuren hebben minder tijd om de kwaliteit van alle content op peil te houden, waardoor je grip en overzicht kunt verliezen. De praktijk wijst uit dat er veel content is die niet meer actueel of relevant is.

Een opgeruimd huis geeft een opgeruimde geest. Je website zou eigenlijk ook jaarlijks een voorjaarsschoonmaak moeten krijgen. Met een content clean up verminder je de hoeveelheid content, wordt het contentbeheer voor je redactie minder arbeidsintensief, en kan de focus weer komen te liggen op kwaliteit en relevantie.
In vier stappen naar minder content
 
Lees ons uitgebreide artikel op Frankwatching (een van de best gelezen artikelen in de maand april).

Wat levert een content clean up op?
Het terugbrengen van je hoeveelheid content levert je veel voordelen op:

Betere beheersbaarheid
Het beheer van je content is na een content clean up minder arbeidsintensief, doordat je minder pagina’s te beheren hebt. Dit levert je financieel voordeel op.

Kwaliteitsverbetering
Doordat je minder content te beheren hebt, neemt de kwaliteit toe. Redacteuren hebben immers meer tijd per pagina. Na de content clean up heb je een helder overzicht van wat je online hebt staan en waarom. Dit heeft grip en overzicht. Hierdoor levert je content meer waarde op.

Hogere gebruiksvriendelijkheid
De gebruiksvriendelijkheid van je website zal toenemen. Je bezoekers vinden gerichter en sneller relevante en actuele content.

Betere vindbaarheid door relevantere content
In de content clean up beoordelen we alle content op relevantie. Door content te verwijderen die geen waarde meer biedt voor je organisatie en je doelgroep, zal de relevantie van je overgebleven content toenemen. Dit maakt je content beter vindbaar en zorgt ervoor dat je beter scoort in zoekmachines.'''
new_tokens = nlp(new_doc)
topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20)
pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary.doc2bow(new_tokens)]], columns=['topic #', 'weight', 'words in topic'])

,topic #,weight,words in topic
0,3,0.35,"0.015*""merk"" + 0.013*""adverteerder"" + 0.012*""k..."
1,4,0.05,"0.025*""televisie"" + 0.025*""nederland"" + 0.023*..."
2,5,0.22,"0.017*""product"" + 0.009*""mens"" + 0.008*""soort""..."
3,11,0.16,"0.024*""ondernemer"" + 0.017*""betalen"" + 0.010*""..."
4,12,0.04,"0.017*""game"" + 0.010*""speler"" + 0.010*""tijd"" +..."
5,14,0.05,"0.011*""brengen"" + 0.008*""online"" + 0.007*""onde..."
6,15,0.06,"0.129*""google"" + 0.017*""dollar"" + 0.016*""yahoo..."
7,18,0.08,"0.020*""site"" + 0.016*""vodafone"" + 0.009*""inter..."


#### Visulizing Topics

In [71]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
# Here a short legend to explain the vis:
# size of bubble: proportional to the proportions of the topics across the N total tokens in the corpus
# red bars: estimated number of times a given term was generated by a given topic
# blue bars: overall frequency of each term in the corpus
# -- Relevance of words is computed with a parameter lambda
# -- Lambda optimal value ~0.6 (https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf)
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary, R=30)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)